# IMPORT LIBRARY

In [0]:
import emotion_config as config
from imagetoarraypreprocessor import ImageToArrayPreprocessor
#from epochcheckpoint import EpochCheckpoint
from trainingmonitor import TrainingMonitor
from emotionvggnet import EmotionVGGNet

In [0]:
import matplotlib
matplotlib.use("Agg")

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import load_model
import keras.backend as K 
import argparse
import os


#Self Imported Library
HDF5 Dataset Generator, because the py file isn't actually working.

In [0]:
#@title HDF5 Dataset Generator
from keras.utils import np_utils
import numpy as np
import h5py

class HDF5DatasetGenerator:
    def __init__(self, dbPath, batchSize, preprocessors = None, aug = None, binarize = True, classes = 2):

        self.batchSize = batchSize
        self.preprocessors = preprocessors
        self.aug = aug
        self.binarize = binarize
        self.classes = classes

        self.db = h5py.File(dbPath)
        self.numImages = self.db["labels"].shape[0]

    def generator(self, passes = np.inf):
        epochs = 0
        while epochs < passes:
            for i in np.arange(0, self.numImages, self.batchSize):

                images = self.db["images"][i: i + self.batchSize]
                labels = self.db["labels"][i: i + self.batchSize]

                if self.binarize:
                    labels = np_utils.to_categorical(labels, self.classes)
                    
                if self.preprocessors is not None:
                    procImages = []
                    for image in images:
                        for p in self.preprocessors:
                            image = p.preprocess(image)
                        procImages.append(image)
                    images = np.array(procImages)
                
                if self.aug is not None:
                    (images, labels) = next(self.aug.flow(images, labels, batch_size = self.batchSize))
                
                yield(images, labels)
            epochs += 1

    def close(self):
        self.db.close()

In [0]:
#@title Epoch Checkpoint
from keras.callbacks import Callback

class EpochCheckpoint(Callback):
    def __init__(self, outputPath, every = 5, startAt = 0):
        super(Callback, self).__init__()

        self.outputPath = outputPath
        self.every = every
        self.intEpoch = startAt

    def on_epoch_end(self, epoch, logs = {}):
        if (self.intEpoch + 1) % self.every == 0:
            p = os.path.sep.join([self.outputPath, "epoch_{}.hdf5".format(self.intEpoch + 1)])
            self.model.save(p, overwrite = True)

        self.intEpoch += 1

# Argument Parsing and Data Generator

In [0]:
ap = argparse.ArgumentParser()
ap.add_argument("-c", "--checkpoints", required = True, help = "path to output checkpoint directory")
ap.add_argument("-m", "--model", type = str, help = "path to *specific* model checkpoint to load")
ap.add_argument("-s", "--start-epoch", type = int, default = 0, help = "epoch to restart training at")
#args = vars(ap.parse_args())
args = vars(ap.parse_args(["--checkpoints", "checkpoints", "--model", "checkpoints/epoch_30.hdf5", "--start-epoch", "30"]))

In [0]:
trainAug = ImageDataGenerator(rotation_range = 10, zoom_range = 0.1, horizontal_flip = True, rescale = 1/255.0, fill_mode = "nearest")
valAug = ImageDataGenerator(rescale = 1/255.0)
iap = ImageToArrayPreprocessor()

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE, aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE, aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)


In [0]:
if args["model"] is None:
    print("[INFO] compiling model...")
    model = EmotionVGGNet.build(width = 48, height = 48, depth = 1, classes = config.NUM_CLASSES)
    opt = Adam(lr = 1e-3)
    model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

else:
    print("[INFO] loading {}...".format(args["model"]))
    model = load_model(args["model"])

    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-3)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading checkpoints/epoch_30.hdf5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 0.0010000000474974513


In [0]:
figPath = os.path.sep.join([config.OUTPUT_PATH, "vggnet_emotion.json"])
jsonPath = os.path.sep.join([config.OUTPUT_PATH, "vggnet_emotion.json"])
callbacks = [
    EpochCheckpoint(args["checkpoints"], every = 5, startAt = args["start_epoch"])
]

# Train the Model

In [0]:
model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // config.BATCH_SIZE,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // config.BATCH_SIZE,
    epochs = 60,
    max_queue_size = config.BATCH_SIZE * 2,
    callbacks = callbacks,
    verbose = 1)

trainGen.close()
valGen.close()

Epoch 1/60
224/224 [==============================] - 20s 90ms/step - loss: 1.0165 - acc: 0.6317 - val_loss: 0.8379 - val_acc: 0.6883
Epoch 2/60
224/224 [==============================] - 15s 66ms/step - loss: 1.0088 - acc: 0.6375 - val_loss: 0.8561 - val_acc: 0.6882
Epoch 3/60
224/224 [==============================] - 15s 67ms/step - loss: 1.0078 - acc: 0.6343 - val_loss: 0.8956 - val_acc: 0.6816
Epoch 4/60
224/224 [==============================] - 15s 65ms/step - loss: 0.9987 - acc: 0.6390 - val_loss: 0.8505 - val_acc: 0.6949
Epoch 5/60
224/224 [==============================] - 14s 65ms/step - loss: 0.9932 - acc: 0.6377 - val_loss: 0.8925 - val_acc: 0.7007
Epoch 6/60
224/224 [==============================] - 11s 48ms/step - loss: 0.9823 - acc: 0.6444 - val_loss: 0.8691 - val_acc: 0.6940
Epoch 7/60
224/224 [==============================] - 11s 47ms/step - loss: 0.9757 - acc: 0.6469 - val_loss: 0.8061 - val_acc: 0.7033
Epoch 8/60
224/224 [==============================] - 11s 48ms